In [ ]:
"""
FANTOM5データを活用した人工プロモーターライブラリ作成システム
元のコードの改良版 + FANTOM5データ統合機能
"""

import pandas as pd
from itertools import product
import requests
from io import StringIO
import re

# ============================================================================
# 基本関数群（元のコードの修正版）
# ============================================================================

def Coplanar(core_promoter, TFB, spacer_length=10):
    """
    DNAの同じ側に必要なタンパク質が結合できるようにする
    """
    if not core_promoter or not TFB:
        raise ValueError('core promoterとTFBは空です')
    spacer = 'N' * spacer_length
    artificial_promoter = TFB + spacer + core_promoter
    return artificial_promoter


def Opposite(core_promoter, TFB, spacer_length=5):
    """
    DNAの反対側に必要なタンパク質が結合できるようにする
    """
    if not core_promoter or not TFB:
        raise ValueError('core promoterとTFBは空です')
    spacer = 'N' * spacer_length
    artificial_promoter = TFB + spacer + core_promoter
    return artificial_promoter


def design_promoter_with_custom_spacer(core_promoter, TFB, spacer_sequence):
    """具体的なspacer配列を指定する"""
    return TFB + spacer_sequence + core_promoter


def validate_sequence(sequence):
    """入力した配列がDNAかどうかを確認する関数"""
    valid_bases = set('ATGCNatgcn')
    return all(base in valid_bases for base in sequence)


def generate_promoter_library(core_csv=None, tfb_csv=None,
                              core_list=None, tfb_list=None,
                              promoter_types=['Coplanar', 'Opposite'],
                              coplanar_spacer=10, opposite_spacer=5,
                              output_csv=None):
    """
    プロモーターライブラリを生成する（修正版）
    """
    # Core promoterの読み込み
    if core_csv:
        core_df = pd.read_csv(core_csv)
        core_col = 'sequence' if 'sequence' in core_df.columns else core_df.columns[0]
        cores = core_df[core_col].tolist()
    elif core_list:
        cores = core_list
    else:
        raise ValueError('core_csvまたはcore_listを指定してください')
    
    # TFBの読み込み（修正：typoを修正）
    if tfb_csv:
        tfb_df = pd.read_csv(tfb_csv)
        tfb_col = 'sequence' if 'sequence' in tfb_df.columns else tfb_df.columns[0]
        tfbs = tfb_df[tfb_col].tolist()
    elif tfb_list:
        tfbs = tfb_list
    else:
        raise ValueError('tfb_csvまたはtfb_listを指定してください')

    # 全ての組み合わせを生成（修正：重複エントリーの問題を解決）
    library = []
    for i, (core, tfb) in enumerate(product(cores, tfbs), 1):
        entry = {
            'ID': f'Promoter_{i:05d}',
            'Core_Promoter': core, 
            'TFB': tfb,
        }

        if 'Coplanar' in promoter_types:
            Coplanar_seq = Coplanar(core, tfb, spacer_length=coplanar_spacer)
            entry['Coplanar_sequence'] = Coplanar_seq
            entry['Coplanar_Length'] = len(Coplanar_seq)

        if 'Opposite' in promoter_types:
            Opposite_seq = Opposite(core, tfb, spacer_length=opposite_spacer)
            entry['Opposite_Sequence'] = Opposite_seq
            entry['Opposite_length'] = len(Opposite_seq)
        
        library.append(entry)  # インデントを修正

    # libraryをDataFrameに変換
    library_df = pd.DataFrame(library)

    # CSVにして出力する
    if output_csv:
        library_df.to_csv(output_csv, index=False)
        print(f'ライブラリを{output_csv}に保存しました')
    
    return library_df


# ============================================================================
# FANTOM5データ統合機能（新機能）
# ============================================================================

class FANTOM5DataHandler:
    """
    FANTOM5データを取得・処理するクラス
    """
    #コントラクタを実行。
    def __init__(self):
        self.base_url = "https://fantom.gsc.riken.jp/5/"

    #メソッドを作る。
    #特定の組織で高発現プロモータ配列を取得。 ここでは便宜上liverになってるけど、実際はメソッド実行時に組織を指定。上位いくつ取るかも、
    # 実際はメソッド実行時に決める。  
    def get_tissue_specific_promoters(self, tissue_type='liver', top_n=100):
        """
        特定組織で高発現のプロモーター配列を取得する（概念的な実装）
        実際のデータアクセスには適切なAPIやダウンロードが必要
        """
        # これは例示的な実装です
        # 実際にはFANTOM5のダウンロードデータを使用する必要があります
        print(f"Note: {tissue_type}組織特異的プロモーターのデモデータを生成中...")
        
        # デモ用のTATA boxとInitiator要素を含むコアプロモーター
        demo_cores = [
            "TATAAA",  # TATA box
            "TATATA",  # TATA box variant
            "YYANWYY", # Initiator (Inr) motif
        ]
        
        return demo_cores[:top_n]
    
    def get_transcription_factor_motifs(self, cell_type='HepG2', top_n=50):
        """
        特定細胞型で活性な転写因子結合モチーフを取得
        """
        # よく知られた転写因子結合モチーフ（例）
        tf_motifs = {
            'SP1': 'GGGCGG',
            'NF-kB': 'GGGACTTTCC',
            'AP-1': 'TGACTCA',
            'CREB': 'TGACGTCA',
            'STAT': 'TTCNNNGAA',
            'HNF4': 'AGGTCA',
            'C/EBP': 'TTGCGCAA',
            'GATA': 'WGATAR',
            'Oct4': 'ATGCAAAT',
            'Sox2': 'CATTGTT',
        }
        
        print(f"Note: {cell_type}細胞型の転写因子モチーフを取得中...")
        return list(tf_motifs.values())[:top_n]
    
    def expand_degenerate_sequences(self, sequence):
        """
        縮重塩基コードを実際の配列に展開
        W = A/T, Y = C/T, R = A/G, N = A/T/G/C
        """
        degenerate_map = {
            'W': ['A', 'T'],
            'Y': ['C', 'T'],
            'R': ['A', 'G'],
            'N': ['A', 'T', 'G', 'C'],
            'S': ['G', 'C'],
            'K': ['G', 'T'],
            'M': ['A', 'C'],
        }
        
        # 縮重塩基を見つける
        for deg_base, alternatives in degenerate_map.items():
            if deg_base in sequence:
                expanded = []
                for alt in alternatives:
                    expanded.extend(
                        self.expand_degenerate_sequences(
                            sequence.replace(deg_base, alt, 1)
                        )
                    )
                return expanded
        
        return [sequence]
    
    def create_promoter_variants(self, core_sequence, n_variants=5):
        """
        コアプロモーター配列のバリアントを作成
        """
        variants = [core_sequence]
        
        # 1塩基置換バリアント
        bases = ['A', 'T', 'G', 'C']
        for i in range(min(len(core_sequence), 2)):  # 最初の2塩基のみ
            for base in bases:
                if base != core_sequence[i]:
                    variant = core_sequence[:i] + base + core_sequence[i+1:]
                    variants.append(variant)
                    if len(variants) >= n_variants:
                        return variants[:n_variants]
        
        return variants[:n_variants]


def create_fantom5_based_library(tissue_type='liver', 
                                 cell_type='HepG2',
                                 n_cores=20,
                                 n_tfbs=15,
                                 include_variants=True,
                                 output_csv='fantom5_promoter_library.csv'):
    """
    FANTOM5データに基づいた人工プロモーターライブラリを作成
    
    Parameters:
    -----------
    tissue_type : str
        対象組織タイプ
    cell_type : str
        対象細胞タイプ
    n_cores : int
        使用するコアプロモーター数
    n_tfbs : int
        使用する転写因子結合部位数
    include_variants : bool
        配列バリアントを含めるか
    output_csv : str
        出力ファイル名
    """
    
    handler = FANTOM5DataHandler()
    
    # 1. FANTOM5から組織特異的コアプロモーターを取得
    print(f"\n=== ステップ1: {tissue_type}組織特異的プロモーターの取得 ===")
    core_promoters = handler.get_tissue_specific_promoters(tissue_type, n_cores)
    
    # 2. 転写因子結合モチーフを取得
    print(f"\n=== ステップ2: {cell_type}細胞型のTFBSモチーフの取得 ===")
    tfbs_motifs = handler.get_transcription_factor_motifs(cell_type, n_tfbs)
    
    # 3. 縮重配列を展開
    print(f"\n=== ステップ3: 縮重配列の展開 ===")
    expanded_cores = []
    for core in core_promoters:
        expanded = handler.expand_degenerate_sequences(core)
        expanded_cores.extend(expanded[:3])  # 各コアから最大3バリアント
    
    expanded_tfbs = []
    for tfb in tfbs_motifs:
        expanded = handler.expand_degenerate_sequences(tfb)
        expanded_tfbs.extend(expanded[:2])  # 各TFBSから最大2バリアント
    
    print(f"展開後: {len(expanded_cores)}個のコアプロモーター")
    print(f"展開後: {len(expanded_tfbs)}個のTFBS")
    
    # 4. プロモーターライブラリ生成
    print(f"\n=== ステップ4: プロモーターライブラリの生成 ===")
    library_df = generate_promoter_library(
        core_list=expanded_cores,
        tfb_list=expanded_tfbs,
        promoter_types=['Coplanar', 'Opposite'],
        coplanar_spacer=10,
        opposite_spacer=5,
        output_csv=output_csv
    )
    
    # 5. 統計情報を追加
    library_df['Tissue_Type'] = tissue_type
    library_df['Cell_Type'] = cell_type
    library_df['GC_Content_Core'] = library_df['Core_Promoter'].apply(
        lambda x: (x.count('G') + x.count('C')) / len(x) * 100
    )
    library_df['GC_Content_TFB'] = library_df['TFB'].apply(
        lambda x: (x.count('G') + x.count('C')) / len(x) * 100
    )
    
    # 再保存
    if output_csv:
        library_df.to_csv(output_csv, index=False)
        print(f"\n最終ライブラリを{output_csv}に保存しました")
    
    print(f"\n=== 完成！ ===")
    print(f"総プロモーター数: {len(library_df)}")
    print(f"ユニークなコア配列: {library_df['Core_Promoter'].nunique()}")
    print(f"ユニークなTFBS配列: {library_df['TFB'].nunique()}")
    
    return library_df


# ============================================================================
# 使用例
# ============================================================================

if __name__ == "__main__":
    
    print("=" * 70)
    print("FANTOM5ベース 人工プロモーターライブラリ作成システム")
    print("=" * 70)
    
    # 例1: 肝臓特異的プロモーターライブラリ
    liver_library = create_fantom5_based_library(
        tissue_type='liver',
        cell_type='HepG2',
        n_cores=10,
        n_tfbs=8,
        output_csv='liver_promoter_library.csv'
    )
    
    print("\n" + "=" * 70)
    print("ライブラリのサンプル:")
    print("=" * 60)
    print(liver_library.head(10))
    
    print("\n" + "=" * 70)
    print("配列長の統計:")
    print("=" * 70)
    if 'Coplanar_Length' in liver_library.columns:
        print(liver_library[['Coplanar_Length', 'Opposite_length']].describe())